### Inference

##### 경로 확인 및 설정

In [70]:
pwd

'd:\\KoBertSum\\src'

In [4]:
cd ./src

d:\KoBertSum\src


##### 필요 라이브러리 불러오기

In [430]:
import copy
import torch
import torch.nn as nn
from transformers import BertModel, BertConfig
from torch.nn.init import xavier_uniform_

from models.model_builder import *
from models.encoder import *

import argparse
import os
import math
import numpy as np

from transformers import BertTokenizer, BertModel
from prepro.tokenization_kobert import *
from prepro.tokenization_kobert import KoBertTokenizer
from kss import split_sentences

##### 인자 설정

In [431]:
def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

parser = argparse.ArgumentParser()
parser.add_argument("-encoder", default='bert', type=str, choices=['bert', 'baseline'])
parser.add_argument("-bert_data_path", default='../bert_data_new/cnndm')
parser.add_argument("-model_path", default='../models/')
parser.add_argument("-result_path", default='../results/cnndm')
parser.add_argument("-temp_dir", default='../temp')

parser.add_argument("-batch_size", default=140, type=int)
parser.add_argument("-test_batch_size", default=200, type=int)

parser.add_argument("-max_pos", default=512, type=int)
parser.add_argument("-use_interval", type=str2bool, nargs='?',const=True,default=True)
parser.add_argument("-large", type=str2bool, nargs='?',const=True,default=False)
parser.add_argument("-load_from_extractive", default='', type=str)

parser.add_argument("-sep_optim", type=str2bool, nargs='?',const=True,default=False)
parser.add_argument("-lr_bert", default=2e-3, type=float)
parser.add_argument("-lr_dec", default=2e-3, type=float)
parser.add_argument("-use_bert_emb", type=str2bool, nargs='?',const=True,default=False)

parser.add_argument("-share_emb", type=str2bool, nargs='?', const=True, default=False)
parser.add_argument("-finetune_bert", type=str2bool, nargs='?', const=True, default=True)
parser.add_argument("-dec_dropout", default=0.2, type=float)
parser.add_argument("-dec_layers", default=6, type=int)
parser.add_argument("-dec_hidden_size", default=768, type=int)
parser.add_argument("-dec_heads", default=8, type=int)
parser.add_argument("-dec_ff_size", default=2048, type=int)
parser.add_argument("-enc_hidden_size", default=512, type=int)
parser.add_argument("-enc_ff_size", default=512, type=int)
parser.add_argument("-enc_dropout", default=0.2, type=float)
parser.add_argument("-enc_layers", default=6, type=int)

parser.add_argument("-pretrained_model", default='bert', type=str)

parser.add_argument("-mode", default='', type=str)
parser.add_argument("-select_mode", default='greedy', type=str)
parser.add_argument("-map_path", default='../../data/')
parser.add_argument("-raw_path", default='../../line_data')
parser.add_argument("-save_path", default='../../data/')

parser.add_argument("-shard_size", default=2000, type=int)
parser.add_argument('-min_src_nsents', default=1, type=int)    # 3
parser.add_argument('-max_src_nsents', default=120, type=int)    # 100
parser.add_argument('-min_src_ntokens_per_sent', default=1, type=int)    # 5
parser.add_argument('-max_src_ntokens_per_sent', default=300, type=int)    # 200
parser.add_argument('-min_tgt_ntokens', default=1, type=int)    # 5
parser.add_argument('-max_tgt_ntokens', default=500, type=int)    # 500

parser.add_argument("-lower", type=str2bool, nargs='?',const=True,default=True)
parser.add_argument("-use_bert_basic_tokenizer", type=str2bool, nargs='?',const=True,default=False)

parser.add_argument('-log_file', default='../../logs/cnndm.log')

parser.add_argument('-dataset', default='')

parser.add_argument('-n_cpus', default=2, type=int)

# params for EXT
parser.add_argument("-ext_dropout", default=0.2, type=float)
parser.add_argument("-ext_layers", default=2, type=int)
parser.add_argument("-ext_hidden_size", default=768, type=int)
parser.add_argument("-ext_heads", default=8, type=int)
parser.add_argument("-ext_ff_size", default=2048, type=int)

parser.add_argument("-label_smoothing", default=0.1, type=float)
parser.add_argument("-generator_shard_size", default=32, type=int)
parser.add_argument("-alpha",  default=0.6, type=float)
parser.add_argument("-beam_size", default=5, type=int)
parser.add_argument("-min_length", default=15, type=int)
parser.add_argument("-max_length", default=150, type=int)
parser.add_argument("-max_tgt_len", default=140, type=int)

args = parser.parse_args('')

##### BertData 변환

In [450]:
class BertData():
    def __init__(self):
        self.tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert", do_lower_case=True)

        self.sep_token = '[SEP]'
        self.cls_token = '[CLS]'
        self.pad_token = '[PAD]'
        self.sep_vid = self.tokenizer.token2idx[self.sep_token]
        self.cls_vid = self.tokenizer.token2idx[self.cls_token]
        self.pad_vid = self.tokenizer.token2idx[self.pad_token]

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [' '.join(s) for s in src]
        idxs = [i for i, s in enumerate(src) if (len(s) > 1)]

        src = [src[i][:2000] for i in idxs]
        src = src[:1000]

        if (len(src) < 3):
            return None

        src_txt = [' '.join(sent) for sent in src]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = self.tokenizer.tokenize(text)
        src_subtokens = src_subtokens[:1022]  ## 512가 최대인데 [SEP], [CLS] 2개 때문에 510
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = None
        src_txt = [original_src_txt[i] for i in idxs]
        tgt_txt = None
        
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt

##### Network

In [451]:
class PositionalEncoding(nn.Module):

    def __init__(self, dropout, dim, max_len=5000):
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, dim, 2, dtype=torch.float) *
                              -(math.log(10000.0) / dim)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)
        pe = pe.unsqueeze(0)
        super(PositionalEncoding, self).__init__()
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
        self.dim = dim

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim)
        if (step):
            emb = emb + self.pe[:, step][:, None, :]

        else:
            emb = emb + self.pe[:, :emb.size(1)]
        emb = self.dropout(emb)
        return emb

    def get_emb(self, emb):
        return self.pe[:, :emb.size(1)]

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attn = MultiHeadedAttention(
            heads, d_model, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, iter, query, inputs, mask):
        if (iter != 0):
            input_norm = self.layer_norm(inputs)
        else:
            input_norm = inputs

        mask = mask.unsqueeze(1)
        context = self.self_attn(input_norm, input_norm, input_norm,
                                 mask=mask)
        out = self.dropout(context) + inputs
        return self.feed_forward(out)

class ExtTransformerEncoder(nn.Module):
    def __init__(self, d_model, d_ff, heads, dropout, num_inter_layers=0):
        super(ExtTransformerEncoder, self).__init__()
        self.d_model = d_model
        self.num_inter_layers = num_inter_layers
        self.pos_emb = PositionalEncoding(dropout, d_model)
        self.transformer_inter = nn.ModuleList(
            [TransformerEncoderLayer(d_model, heads, d_ff, dropout)
             for _ in range(num_inter_layers)])
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.wo = nn.Linear(d_model, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, top_vecs, mask):
        """ See :obj:`EncoderBase.forward()`"""

        batch_size, n_sents = top_vecs.size(0), top_vecs.size(1)
        pos_emb = self.pos_emb.pe[:, :n_sents]
        x = top_vecs * mask[:, :, None].float()
        x = x + pos_emb

        for i in range(self.num_inter_layers):
            x = self.transformer_inter[i](i, x, x, ~mask)  # all_sents * max_tokens * dim

        x = self.layer_norm(x)
        sent_scores = self.sigmoid(self.wo(x))
        sent_scores = sent_scores.squeeze(-1) * mask.float()

        return sent_scores
    
class Bert(nn.Module):
    temp_dir = 'D:/KoBertSum/temp'
    def __init__(self, large, temp_dir, finetune=False):
        super(Bert, self).__init__()
        self.model = BertModel.from_pretrained("monologg/kobert", cache_dir=temp_dir)

        self.finetune = finetune

    def forward(self, x, segs, mask):
        top_vec = self.model(input_ids=x, token_type_ids=segs, attention_mask=mask)[0]

        return top_vec
    
class ExtSummarizer(nn.Module):
    def __init__(self, args, device, checkpoint):
        super(ExtSummarizer, self).__init__()
        self.args = args
        self.device = device
        self.bert = Bert(args.large, args.temp_dir)

        self.ext_layer = ExtTransformerEncoder(self.bert.model.config.hidden_size, args.ext_ff_size, args.ext_heads,
                                               args.ext_dropout, args.ext_layers)
        if (args.encoder == 'baseline'):
            bert_config = BertConfig(self.bert.model.config.vocab_size, hidden_size=args.ext_hidden_size,
                                     num_hidden_layers=args.ext_layers, num_attention_heads=args.ext_heads, intermediate_size=args.ext_ff_size)
            self.bert.model = BertModel(bert_config)
            self.ext_layer = Classifier(self.bert.model.config.hidden_size)

        if(args.max_pos>512):
            my_pos_embeddings = nn.Embedding(args.max_pos, self.bert.model.config.hidden_size)
            my_pos_embeddings.weight.data[:512] = self.bert.model.embeddings.position_embeddings.weight.data
            my_pos_embeddings.weight.data[512:] = self.bert.model.embeddings.position_embeddings.weight.data[-1][None,:].repeat(args.max_pos-512,1)
            self.bert.model.embeddings.position_embeddings = my_pos_embeddings


        if checkpoint is not None:
            self.load_state_dict(checkpoint['model'], strict=True)
        else:
            if args.param_init != 0.0:
                for p in self.ext_layer.parameters():
                    p.data.uniform_(-args.param_init, args.param_init)
            if args.param_init_glorot:
                for p in self.ext_layer.parameters():
                    if p.dim() > 1:
                        xavier_uniform_(p)

        self.to(device)

    def forward(self, src, segs, clss, mask_src, mask_cls):
        top_vec = self.bert(src, segs, mask_src)
        #print(top_vec)
        #top_vec = top_vec.last_hidden_state
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()
        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)
        return sent_scores, mask_cls

In [452]:
def summarize(text):
    
    def txt2input(text):
        #data = list(filter(None, text.split('\n')))
        #data = split_sentences(text)
        bertdata = BertData()
        txt_data = bertdata.preprocess(text)
        data_dict = {"src":txt_data[0],
                    "labels":[0,1,2],
                    "segs":txt_data[2],
                    "clss":txt_data[3],
                    "src_txt":txt_data[4],
                    "tgt_txt":None}
        input_data = []
        input_data.append(data_dict)
        return input_data
    
    input_data = txt2input(text)
    device = torch.device("cuda")
    
    def _pad(data, pad_id, width=-1):
        if (width == -1):
            width = max(len(d) for d in data)
        rtn_data = [d + [pad_id] * (width - len(d)) for d in data]
        return rtn_data
    
    pre_src = [x['src'] for x in input_data]
    pre_segs = [x['segs'] for x in input_data]
    pre_clss = [x['clss'] for x in input_data]

    src = torch.tensor(_pad(pre_src, 0)).cuda()
    segs = torch.tensor(_pad(pre_segs, 0)).cuda()
    mask_src = ~(src == 0)

    clss = torch.tensor(_pad(pre_clss, -1)).cuda()
    mask_cls = ~(clss == -1)
    clss[clss == -1] = 0

    clss.to(device).long()
    mask_cls.to(device).long()
    segs.to(device).long()
    mask_src.to(device).long()
    
    #checkpoint = torch.load("D:/KoBertSum/ext/models/model_step_26000.pt")  # V2
    #checkpoint = torch.load("D:/KoBertSum/ext/models/model_step_5000.pt")  # V1
    checkpoint = torch.load("D:/KoBertSum/ext/models/model_step_5000_2.pt")  # V3 - max_pos 1024
    model = ExtSummarizer(args, device, checkpoint)
    model.eval()

    with torch.no_grad():
        sent_scores, mask = model(src, segs, clss, mask_src, mask_cls)
        sent_scores = sent_scores + mask.float()
        sent_scores = sent_scores.cpu().data.numpy()
        print(sent_scores)
        selected_ids = np.argsort(-sent_scores, 1)
        print(selected_ids)
    
    return selected_ids

##### 결과 확인

In [439]:
text = '''
"뭔가 있긴 있는 거 같은데, 아직은 완벽하지 않은 것 같다."

국내 연구진이 개발한 상온 초전도체 'LK-99'의 정체에 대해 국내 과학자들은 말끝을 흐리며 이같이 애매모호하게 답했다. 국내외에서 재현 실험 등의 검증이 이뤄지고 있는 상황에서 확실하게 말할 수 없음을 이해해 달라고도 했다. 그럼에도 이전까지 볼 수 없었던 물질임에는 틀림 없다는 여운을 남겼다.

취재 과정에서 인터뷰한 대다수 과학자들은 LK-99가 초전도 특성을 가진 물질인 거 같긴 한데, 그렇다고 100% 초전도체라고 현재로선 단정적으로 말하기 어렵다는 얘기를 공통적으로 꺼냈다.

초전도체 검증 결과 여부를 떠나 21세기 들어 이처럼 새로운 과학적 발명에 이토록 전 세계가 떠들석하고, 가히 센세이셔널한 반응을 몰고 온 적이 있었던가. 아마도 현대과학의 혁명기로, 과학적 발명과 발견이 많았던 19세기 말과 20세기 초를 제외하곤 이번이 처음인 듯 싶다. 그만큼 인류가 얼마나 세기적 발명의 탄생을 목말라 했는지를 가늠케 한 사건이었다.

지난달 22일 퀀텀에너지연구소가 상온·상압 초전도체 'LK-99' 개발에 대한 논문을 사전 공개 사이트(아카이브)에 올린 이후 지금까지 전 세계가 우리나라를 주목하고 있다. 112년 전인 1911년 네덜란드 라이덴 대학의 카멜린 온네스 교수가 초전도 현상을 발견한 이후, 상온 초전도체 개발에 수많은 과학자들이 밤낮을 잊고 실험실을 지켜 왔지만 모두 허사로 끝났다.

이렇듯 '꿈의 물질'로 불리는 상온 초전도체 개발은 노벨상 '0순위'이자, 기존 기술패권 질서의 판을 한 순간에 뒤집어 놓을 파괴적 혁신기술로 추앙 받아왔다. 그토록 갈망하던 현대 과학계의 난제 중의 하나인 상온 초전도체 개발 소식이 한국에서 전해진 것이다.

그것도 세계 유수의 대학과 연구소, 글로벌 대기업이 아닌 2008년 창업한 대학 실험실 기반 벤처기업이 깜짝 주인공으로 등장했다. 이 기업은 국내 상온·상압 초전도체 연구의 토대를 놓은 초전도 이론의 대가로 불리는 고(故) 최동식 고려대 교수의 제자들이 주축이 돼 설립됐다. 이들의 논문에 '고 최동식 교수를 기립니다'는 문장이 기재돼 있을 정도로, 대학 시절부터 20년 넘게 스승의 유훈을 받들어 상온 초전도체 개발을 이어 왔다.

LK-99는 이석배(L) 대표와 김지훈(K) 연구소장의 영문 이니셜을 따서 정했고, LK-99를 발견해 본격적인 연구를 시작한 1999년을 기념해 '99'라고 붙여졌다. 이들은 지난 20년 동안 1000회 이상의 실험을 반복한 끝에 LK-99 개발에 성공했다고 했다.

LK-99 진위 여부를 확인하기 위해 전 세계 주류 연구그룹들이 일을 제쳐놓고 재현 실험 등을 통한 검증에 힘을 쏟고 있다. 글로벌 빅테크 기업도 이에 합류했다는 소식도 들린다. 지금까지의 결과를 보면 LK-99는 상온 초전도체가 아닐 가능성이 커 보인다. 더 많은 검증 결과가 나올수록 한국발 '상온 초전도체 탄생'은 물거품이 될 가능성이 높아 보인다. 미 프린스턴대는 LK-99가 자석일 가능성이 크다고 밝혔다.

상온 초전도체에 대한 회의적 시각이 늘면서 LK-99 검증 논란을 '제2의 황우석 사태'에 빗대어 깎아 내릴 듯 하다. 분명한 것은 논문을 고의로 조작한 황우석 사태와 LK-99는 본질적으로 다르다. 이들의 논문 데이터가 다소 부실해 보여도 데이터 조작 흔적은 없다는 게 과학자들의 공통된 주장이다.

여기에 줄기세포 논란에 거짓 대응한 황우석과 달리 이들은 과학계의 검증에 열린 마음으로 대응하고 있다는 점도 커다란 차이다. 비록 LK-99가 상온 초전도체가 아니더라도 우리가 떳떳하지 못할 이유가 없다. 이전에 발견하지 못했던 새로운 물질이라는 점만 입증돼도 이들의 노력은 충분히 높게 평가 받아야 한다. 결코 황우석 사태와 비교해 비난해선 안 될 것이다.

과학은 무수한 실패 과정을 거쳐 진보해 왔다. 그렇기에 과학은 솔직해야 하고, 거짓이 없어야 한다. LK-99가 신기루에 불과하더라도 우리의 작은 벤처가 촉발시킨 상온 초전도체 이슈는 전 세계 과학자들의 개발 경쟁을 가속화해 우리를 더 나은 세상으로 이끌어 주는 트리거가 될 것이라는 점만으로도 의미있는 일이 아닐까. 우리나라가 초전도체 개발을 계속 이어가야 하는 이유이기도 하다.
'''

In [440]:
import kss
text = kss.split_sentences(text)
text

['"뭔가 있긴 있는 거 같은데, 아직은 완벽하지 않은 것 같다."',
 "국내 연구진이 개발한 상온 초전도체 'LK-99'의 정체에 대해 국내 과학자들은 말끝을 흐리며 이같이 애매모호하게 답했다.",
 '국내외에서 재현 실험 등의 검증이 이뤄지고 있는 상황에서 확실하게 말할 수 없음을 이해해 달라고도 했다.',
 '그럼에도 이전까지 볼 수 없었던 물질임에는 틀림 없다는 여운을 남겼다.',
 '취재 과정에서 인터뷰한 대다수 과학자들은 LK-99가 초전도 특성을 가진 물질인 거 같긴 한데, 그렇다고 100% 초전도체라고 현재로선 단정적으로 말하기 어렵다는 얘기를 공통적으로 꺼냈다.',
 '초전도체 검증 결과 여부를 떠나 21세기 들어 이처럼 새로운 과학적 발명에 이토록 전 세계가 떠들석하고, 가히 센세이셔',
 '널한 반응을 몰고 온 적이 있었던가.',
 '아마도 현대과학의 혁명기로, 과학적 발명과 발견이 많았던 19세기 말과 20세기 초를 제외하곤 이번이 처음인 듯 싶다.',
 '그만큼 인류가 얼마나 세기적 발명의 탄생을 목말라 했는지를 가늠케 한 사건이었다.',
 "지난달 22일 퀀텀에너지연구소가 상온·상압 초전도체 'LK-99' 개발에 대한 논문을 사전 공개 사이트(아카이브)에 올린 이후 지금까지 전 세계가 우리나라를 주목하고 있다.",
 '112년 전인 1911년 네덜란드 라이덴 대학의 카멜린 온네스 교수가 초전도 현상을 발견한 이후, 상온 초전도체 개발에 수많은 과학자들이 밤낮을 잊고 실험실을 지켜 왔지만 모두 허사로 끝났다.',
 "이렇듯 '꿈의 물질'로 불리는 상온 초전도체 개발은 노벨상 '0순위'이자, 기존 기술패권 질서의 판을 한 순간에 뒤집어 놓을 파괴적 혁신기술로 추앙 받아왔다.",
 '그토록 갈망하던 현대 과학계의 난제 중의 하나인 상온 초전도체 개발 소식이 한국에서 전해진 것이다.',
 '그것도 세계 유수의 대학과 연구소, 글로벌 대기업이 아닌 2008년 창업한 대학 실험실 기반 벤처기업이 깜짝 주인공으로 등장했다.',
 '이 기업은 국내

In [453]:
text = ['(서울=연합뉴스) 이주영 기자 = 빠른 진화 속도로 혹독한 환경에 적응하며 4억년을 살아온 히말라야 티베트고원의 이끼도 현재 진행되고 있는 지구온난화에는 살아남지 못할 것이라는 연구 결과가 나왔다.',
"독일 프라이부르크대 랄프 레스키 교수와 중국 서우두사범대 허이쿤 교수팀은 10일 과학저널 '셀'(Cell)에서 티베트고원 등에 사는 화석 식물인 타카키아 이끼의 DNA 분석 결과 유전적으로 매우 빠른 진화 특성을 가졌지만 현 기후변화에서 살아남을 만큼 빠르게 진화하지는 못할 가능성이 큰 것으로 나타났다고 밝혔다.",
'티베트고원 얼음 절벽에서 3억9천만년이나 살아온 타카키아는 작고 느리게 자라는 이끼로 히말라야 4천ｍ 고지대와 일본, 미국 등 일부 지역에서만 볼 수 있다.',
'연구팀은 티베트고원의 타카키아 서식지를 10년간 18차례 방문해 샘플을 수집하고 서식지를 조사했다.',
'타카키아의 DNA 염기서열을 분석하고 기후변화가 타카키아에 어떤 영향을 미치는지 연구했다.',
'레스키 교수는 지각변동으로 히말라야산맥이 솟아올랐을 때는 타카키아가 등장한 지 1억년이 지난 시점이었고, 이런 급격한 환경 변화 속에서 살아남았다며 이 연구를 통해 그 비밀을 밝히고자 했다고 말했다.',
'연구팀은 DNA 분석 결과 타카키아의 게놈(유전체)이 여러 세대에 걸쳐 자연선택에 의해 진화하면서 손상된 DNA를 고치고 자외선 손상으로부터 회복하는 데 탁월한 유전자들을 많이 갖게 된 것으로 나타났다고 밝혔다.',
'레스키 교스는 "타카키아가 현재 빠르게 진화하는 유전자가 가장 많은 게놈을 가지고 있다는 사실을 발견했다"고 말했다.',
'허이쿤 교수는 "타카키아는 매년 8개월간 눈에 덮여 있고 4개월은 고강도 자외선을 받는다"며 "타카키아는 이에 대응해 유연한 가지 뻗기로 다양한 위치에서 살 수 있게 적응했고 이를 통해 폭설과 자외선을 견딜 수 있는 견고한 개체군 구조를 만들었다"고 설명했다.',
'연구팀은 또 타카키아 분류에 대해서도 이끼인지, 조류인지 등에 대한 논란이 있었으나 이번 게놈 분석으로 이끼라는 게 확인됐다며 시간 흐름에 따라 게놈이 크게 변했음에도 식물체 형태가 거의 변하지 않은 점은 새 연구 과제라고 밝혔다.',
'연구팀은 그러나 타카키아가 과거 환경변화에 빠르게 적응해 살아남았지만 현재의 온난화와 서식지 감소 속도를 고려하면 앞으로 100년 이상 살아남기는 어려울 것으로 보인다고 우려했다.',
'연구가 진행되는 동안 티베트고원의 타카키아 개체수는 매년 1.6%씩 감소했으며 서식지도 빠르게 줄어 금세기 말에는 타카키아에 적합한 서식지가 세계적으로 1천~1천500㎢밖에 남지 않을 것으로 예상됐다.',
'연구팀은 타카키아의 멸종을 막기 위해 실험실에서 타카키아를 증식한 다음 티베트고원에 이식하는 시도를 하고 있으며 5년간 관찰 결과 이식된 식물 일부가 생존하고 번성하는 것으로 나타났다고 밝혔다.',
'레스키 교수는 "인간이 진화 정점에 있다고 생각하지만 공룡도 왔다가 사라진 것처럼 인간도 사라질 수 있다"며 "공룡의 등장과 멸종, 인간의 등장을 지켜본 타카키아로부터 회복력과 멸종에 대해 무언가를 배울 수 있을 것"이라고 말했다.']

In [454]:
result = summarize(text)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


RuntimeError: The size of tensor a (1024) must match the size of tensor b (512) at non-singleton dimension 1

In [449]:
result = summarize(text)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


[[1.2912345 1.35289   1.3499987 1.3547711 1.2963245 1.2898964]]
[[3 1 2 4 0 5]]


In [445]:
[text[i] for i in result[0][:len(text)//3]]
#[text[i] for i in result[0][:3]]

['연구팀은 티베트고원의 타카키아 서식지를 10년간 18차례 방문해 샘플을 수집하고 서식지를 조사했다.',
 "독일 프라이부르크대 랄프 레스키 교수와 중국 서우두사범대 허이쿤 교수팀은 10일 과학저널 '셀'(Cell)에서 티베트고원 등에 사는 화석 식물인 타카키아 이끼의 DNA 분석 결과 유전적으로 매우 빠른 진화 특성을 가졌지만 현 기후변화에서 살아남을 만큼 빠르게 진화하지는 못할 가능성이 큰 것으로 나타났다고 밝혔다.",
 '티베트고원 얼음 절벽에서 3억9천만년이나 살아온 타카키아는 작고 느리게 자라는 이끼로 히말라야 4천ｍ 고지대와 일본, 미국 등 일부 지역에서만 볼 수 있다.',
 '타카키아의 DNA 염기서열을 분석하고 기후변화가 타카키아에 어떤 영향을 미치는지 연구했다.']

##### 탐구

In [468]:
text = ['BNK경남은행 간부의 562억원 횡령 사건이 채 잊히기도 전인데 또 다른 금융사고가 연발하고 있다.',
'수법도 거래 기업의 미공개 정보로 주식을 거래해 부당이득을 취하고, 고객 몰래 계좌를 개설하는 등 눈앞의 이익을 위해 온갖 불법·편법 행위가 동원되고 있다.',
'KB국민은행의 직원들이 고객사의 미공개정보를 이용해 주식을 사고팔아 127억원의 부당이득을 챙긴 사실이 최근 금융당국에 적발됐다.',
'이들은 2021년 1월부터 올해 4월까지 61개 상장사의 증권 업무를 대행하며 알게 된 무상증자 규모와 일정을 주식 매수에 이용했다.',
'정보 공개 전 미리 주식을 사뒀다가 공시 뒤 주가가 오르면 팔았다.',
'무상증자를 하게 되면 기업재무구조가 건전한 것으로 풀이돼 주가에는 호재로 작용한다.',
'이런 방식으로 66억원 정도를 챙겼고, 일부는 다른 부서의 동료나 가족 등에게도 정보를 전달했다.',
'이 과정에서도 61억원 상당의 부당 이득이 발생했다.',
'대구은행 일부 지점 직원 수십명은 평가 실적을 올리기 위해 지난해 1000여건이 넘는 고객 문서를 위조해 증권 계좌를 개설한 것으로 파악됐다.',
'이 직원들은 내점한 고객을 상대로 증권사 연계 계좌를 만들어 달라고 요청한 뒤 해당 계좌 신청서를 복사해 고객의 동의 없이 같은 증권사의 계좌를 하나 더 만들었다.',
'A증권사 위탁 계좌 개설 신청서를 받고, 같은 신청서를 복사해 A증권사 해외선물계좌까지 개설하는 방식이다.',
'최근 한 고객이 동의하지 않은 계좌가 개설됐다는 사실을 알게 돼 대구은행에 민원을 제기하면서 직원들의 비리가 드러났다.',
'대구은행은 문제를 인지하고도 금감원에 이 사실을 보고하지 않았고, 영업점들에 공문을 보내 불건전 영업행위를 예방하라고 안내하는 데 그쳤다.',
'금융실명제법 위반, 사문서 위조 등에 해당할 수 있는 범죄행위를 대수롭지 않게 넘기는 안일함이 혀를 차게 한다.',
'국내 은행은 땅 짚고 헤엄치기식 이자장사로 평균 1억원대 고연봉을 누리는 직종이다.',
'시중은행은 미국발 고금리에 편승해 거둬들인 막대한 예대마진으로 최근 수년간 성과급 잔치를 벌여 국민의 눈총을 받았다.',
'국민의 재산으로 손쉽게 수익을 올리는 직종이라면 누구보다 엄격한 도덕적 기준을 세워도 모자랄 판에 ‘내 몫을 더 챙기겠다’며 이기적 탐욕을 부리고 있으니 말문이 막힌다.',
'자체 내부통제가 안 된다면 현행 솜방망이 처벌 수위를 높이는 수밖에 없다.',
'주요 동기가 경제적 이익인 만큼 벌금이나 과징금, 양형 부과 수준을 크게 높여 법 무서운 줄 알도록 해야 한다.',
'주요 선진국에서 이미 도입한 불공정거래 범죄자에 대한 자본시장 거래제한제도도 적극 검토할 필요가 있다.']

In [365]:
import kss
text = kss.split_sentences(text)
text

['(서울=연합뉴스) 이주영 기자 = 빠른 진화 속도로 혹독한 환경에 적응하며 4억년을 살아온 히말라야 티베트고원의 이끼도 현재 진행되고 있는 지구온난화에는 살아남지 못할 것이라는 연구 결과가 나왔다.',
 "독일 프라이부르크대 랄프 레스키 교수와 중국 서우두사범대 허이쿤 교수팀은 10일 과학저널 '셀'(Cell)에서 티베트고원 등에 사는 화석 식물인 타카키아 이끼의 DNA 분석 결과 유전적으로 매우 빠른 진화 특성을 가졌지만 현 기후변화에서 살아남을 만큼 빠르게 진화하지는 못할 가능성이 큰 것으로 나타났다고 밝혔다.",
 '티베트고원 얼음 절벽에서 3억9천만년이나 살아온 타카키아는 작고 느리게 자라는 이끼로 히말라야 4천ｍ 고지대와 일본, 미국 등 일부 지역에서만 볼 수 있다.',
 '연구팀은 티베트고원의 타카키아 서식지를 10년간 18차례 방문해 샘플을 수집하고 서식지를 조사했다.',
 '타카키아의 DNA 염기서열을 분석하고 기후변화가 타카키아에 어떤 영향을 미치는지 연구했다.',
 '레스키 교수는 지각변동으로 히말라야산맥이 솟아올랐을 때는 타카키아가 등장한 지 1억년이 지난 시점이었고, 이런 급격한 환경 변화 속에서 살아남았다며 이 연구를 통해 그 비밀을 밝히고자 했다고 말했다.',
 '연구팀은 DNA 분석 결과 타카키아의 게놈(유전체)이 여러 세대에 걸쳐 자연선택에 의해 진화하면서 손상된 DNA를 고치고 자외선 손상으로부터 회복하는 데 탁월한 유전자들을 많이 갖게 된 것으로 나타났다고 밝혔다.',
 '레스키 교스는 "타카키아가 현재 빠르게 진화하는 유전자가 가장 많은 게놈을 가지고 있다는 사실을 발견했다"고 말했다.',
 '허이쿤 교수는 "타카키아는 매년 8개월간 눈에 덮여 있고 4개월은 고강도 자외선을 받는다"며 "타카키아는 이에 대응해 유연한 가지 뻗기로 다양한 위치에서 살 수 있게 적응했고 이를 통해 폭설과 자외선을 견딜 수 있는 견고한 개체군 구조를 만들었다"고 설명했다.',
 '연구팀은 또 타카키아 분류에 대해서도 이끼인지, 조류인지 등에 대한

In [416]:
src = []
src.append(text)
src

[['BNK경남은행 간부의 562억원 횡령 사건이 채 잊히기도 전인데 또 다른 금융사고가 연발하고 있다.',
  '수법도 거래 기업의 미공개 정보로 주식을 거래해 부당이득을 취하고, 고객 몰래 계좌를 개설하는 등 눈앞의 이익을 위해 온갖 불법·편법 행위가 동원되고 있다.',
  'KB국민은행의 직원들이 고객사의 미공개정보를 이용해 주식을 사고팔아 127억원의 부당이득을 챙긴 사실이 최근 금융당국에 적발됐다.',
  '이들은 2021년 1월부터 올해 4월까지 61개 상장사의 증권 업무를 대행하며 알게 된 무상증자 규모와 일정을 주식 매수에 이용했다.',
  '정보 공개 전 미리 주식을 사뒀다가 공시 뒤 주가가 오르면 팔았다.',
  '무상증자를 하게 되면 기업재무구조가 건전한 것으로 풀이돼 주가에는 호재로 작용한다.',
  '이런 방식으로 66억원 정도를 챙겼고, 일부는 다른 부서의 동료나 가족 등에게도 정보를 전달했다.',
  '이 과정에서도 61억원 상당의 부당 이득이 발생했다.',
  '대구은행 일부 지점 직원 수십명은 평가 실적을 올리기 위해 지난해 1000여건이 넘는 고객 문서를 위조해 증권 계좌를 개설한 것으로 파악됐다.',
  '이 직원들은 내점한 고객을 상대로 증권사 연계 계좌를 만들어 달라고 요청한 뒤 해당 계좌 신청서를 복사해 고객의 동의 없이 같은 증권사의 계좌를 하나 더 만들었다.',
  'A증권사 위탁 계좌 개설 신청서를 받고, 같은 신청서를 복사해 A증권사 해외선물계좌까지 개설하는 방식이다.',
  '최근 한 고객이 동의하지 않은 계좌가 개설됐다는 사실을 알게 돼 대구은행에 민원을 제기하면서 직원들의 비리가 드러났다.',
  '대구은행은 문제를 인지하고도 금감원에 이 사실을 보고하지 않았고, 영업점들에 공문을 보내 불건전 영업행위를 예방하라고 안내하는 데 그쳤다.',
  '금융실명제법 위반, 사문서 위조 등에 해당할 수 있는 범죄행위를 대수롭지 않게 넘기는 안일함이 혀를 차게 한다.',
  '국내 은행은 땅 짚고 헤엄치기식 이자장사로 평

In [456]:
original_src_txt = [' '.join(s) for s in text]
original_src_txt

['B N K 경 남 은 행   간 부 의   5 6 2 억 원   횡 령   사 건 이   채   잊 히 기 도   전 인 데   또   다 른   금 융 사 고 가   연 발 하 고   있 다 .',
 '수 법 도   거 래   기 업 의   미 공 개   정 보 로   주 식 을   거 래 해   부 당 이 득 을   취 하 고 ,   고 객   몰 래   계 좌 를   개 설 하 는   등   눈 앞 의   이 익 을   위 해   온 갖   불 법 · 편 법   행 위 가   동 원 되 고   있 다 .',
 'K B 국 민 은 행 의   직 원 들 이   고 객 사 의   미 공 개 정 보 를   이 용 해   주 식 을   사 고 팔 아   1 2 7 억 원 의   부 당 이 득 을   챙 긴   사 실 이   최 근   금 융 당 국 에   적 발 됐 다 .',
 '이 들 은   2 0 2 1 년   1 월 부 터   올 해   4 월 까 지   6 1 개   상 장 사 의   증 권   업 무 를   대 행 하 며   알 게   된   무 상 증 자   규 모 와   일 정 을   주 식   매 수 에   이 용 했 다 .',
 '정 보   공 개   전   미 리   주 식 을   사 뒀 다 가   공 시   뒤   주 가 가   오 르 면   팔 았 다 .',
 '무 상 증 자 를   하 게   되 면   기 업 재 무 구 조 가   건 전 한   것 으 로   풀 이 돼   주 가 에 는   호 재 로   작 용 한 다 .',
 '이 런   방 식 으 로   6 6 억 원   정 도 를   챙 겼 고 ,   일 부 는   다 른   부 서 의   동 료 나   가 족   등 에 게 도   정 보 를   전 달 했 다 .',
 '이   과 정 에 서 도   6 1 억 원   상 당 의   부 당   이 득 이   발 생 했 다 .',
 '대 구 은 행   일 부   지 점   직 원   수 십 명 은   평 가   실 적 을   올 리 기 

In [457]:
idxs = [i for i, s in enumerate(text) if (len(s) > 1)]
idxs

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [458]:
text = [text[i][:2000] for i in idxs]
text

['BNK경남은행 간부의 562억원 횡령 사건이 채 잊히기도 전인데 또 다른 금융사고가 연발하고 있다.',
 '수법도 거래 기업의 미공개 정보로 주식을 거래해 부당이득을 취하고, 고객 몰래 계좌를 개설하는 등 눈앞의 이익을 위해 온갖 불법·편법 행위가 동원되고 있다.',
 'KB국민은행의 직원들이 고객사의 미공개정보를 이용해 주식을 사고팔아 127억원의 부당이득을 챙긴 사실이 최근 금융당국에 적발됐다.',
 '이들은 2021년 1월부터 올해 4월까지 61개 상장사의 증권 업무를 대행하며 알게 된 무상증자 규모와 일정을 주식 매수에 이용했다.',
 '정보 공개 전 미리 주식을 사뒀다가 공시 뒤 주가가 오르면 팔았다.',
 '무상증자를 하게 되면 기업재무구조가 건전한 것으로 풀이돼 주가에는 호재로 작용한다.',
 '이런 방식으로 66억원 정도를 챙겼고, 일부는 다른 부서의 동료나 가족 등에게도 정보를 전달했다.',
 '이 과정에서도 61억원 상당의 부당 이득이 발생했다.',
 '대구은행 일부 지점 직원 수십명은 평가 실적을 올리기 위해 지난해 1000여건이 넘는 고객 문서를 위조해 증권 계좌를 개설한 것으로 파악됐다.',
 '이 직원들은 내점한 고객을 상대로 증권사 연계 계좌를 만들어 달라고 요청한 뒤 해당 계좌 신청서를 복사해 고객의 동의 없이 같은 증권사의 계좌를 하나 더 만들었다.',
 'A증권사 위탁 계좌 개설 신청서를 받고, 같은 신청서를 복사해 A증권사 해외선물계좌까지 개설하는 방식이다.',
 '최근 한 고객이 동의하지 않은 계좌가 개설됐다는 사실을 알게 돼 대구은행에 민원을 제기하면서 직원들의 비리가 드러났다.',
 '대구은행은 문제를 인지하고도 금감원에 이 사실을 보고하지 않았고, 영업점들에 공문을 보내 불건전 영업행위를 예방하라고 안내하는 데 그쳤다.',
 '금융실명제법 위반, 사문서 위조 등에 해당할 수 있는 범죄행위를 대수롭지 않게 넘기는 안일함이 혀를 차게 한다.',
 '국내 은행은 땅 짚고 헤엄치기식 이자장사로 평균 1억원대 고연봉을 누리는

In [459]:
text = text[:1000]
len(text)

20

In [422]:
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert", do_lower_case=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [460]:
src_txt = [' '.join(sent) for sent in text]
src_txt

['B N K 경 남 은 행   간 부 의   5 6 2 억 원   횡 령   사 건 이   채   잊 히 기 도   전 인 데   또   다 른   금 융 사 고 가   연 발 하 고   있 다 .',
 '수 법 도   거 래   기 업 의   미 공 개   정 보 로   주 식 을   거 래 해   부 당 이 득 을   취 하 고 ,   고 객   몰 래   계 좌 를   개 설 하 는   등   눈 앞 의   이 익 을   위 해   온 갖   불 법 · 편 법   행 위 가   동 원 되 고   있 다 .',
 'K B 국 민 은 행 의   직 원 들 이   고 객 사 의   미 공 개 정 보 를   이 용 해   주 식 을   사 고 팔 아   1 2 7 억 원 의   부 당 이 득 을   챙 긴   사 실 이   최 근   금 융 당 국 에   적 발 됐 다 .',
 '이 들 은   2 0 2 1 년   1 월 부 터   올 해   4 월 까 지   6 1 개   상 장 사 의   증 권   업 무 를   대 행 하 며   알 게   된   무 상 증 자   규 모 와   일 정 을   주 식   매 수 에   이 용 했 다 .',
 '정 보   공 개   전   미 리   주 식 을   사 뒀 다 가   공 시   뒤   주 가 가   오 르 면   팔 았 다 .',
 '무 상 증 자 를   하 게   되 면   기 업 재 무 구 조 가   건 전 한   것 으 로   풀 이 돼   주 가 에 는   호 재 로   작 용 한 다 .',
 '이 런   방 식 으 로   6 6 억 원   정 도 를   챙 겼 고 ,   일 부 는   다 른   부 서 의   동 료 나   가 족   등 에 게 도   정 보 를   전 달 했 다 .',
 '이   과 정 에 서 도   6 1 억 원   상 당 의   부 당   이 득 이   발 생 했 다 .',
 '대 구 은 행   일 부   지 점   직 원   수 십 명 은   평 가   실 적 을   올 리 기 

In [461]:
text = ' [SEP] [CLS] '.join(src_txt)
text

'B N K 경 남 은 행   간 부 의   5 6 2 억 원   횡 령   사 건 이   채   잊 히 기 도   전 인 데   또   다 른   금 융 사 고 가   연 발 하 고   있 다 . [SEP] [CLS] 수 법 도   거 래   기 업 의   미 공 개   정 보 로   주 식 을   거 래 해   부 당 이 득 을   취 하 고 ,   고 객   몰 래   계 좌 를   개 설 하 는   등   눈 앞 의   이 익 을   위 해   온 갖   불 법 · 편 법   행 위 가   동 원 되 고   있 다 . [SEP] [CLS] K B 국 민 은 행 의   직 원 들 이   고 객 사 의   미 공 개 정 보 를   이 용 해   주 식 을   사 고 팔 아   1 2 7 억 원 의   부 당 이 득 을   챙 긴   사 실 이   최 근   금 융 당 국 에   적 발 됐 다 . [SEP] [CLS] 이 들 은   2 0 2 1 년   1 월 부 터   올 해   4 월 까 지   6 1 개   상 장 사 의   증 권   업 무 를   대 행 하 며   알 게   된   무 상 증 자   규 모 와   일 정 을   주 식   매 수 에   이 용 했 다 . [SEP] [CLS] 정 보   공 개   전   미 리   주 식 을   사 뒀 다 가   공 시   뒤   주 가 가   오 르 면   팔 았 다 . [SEP] [CLS] 무 상 증 자 를   하 게   되 면   기 업 재 무 구 조 가   건 전 한   것 으 로   풀 이 돼   주 가 에 는   호 재 로   작 용 한 다 . [SEP] [CLS] 이 런   방 식 으 로   6 6 억 원   정 도 를   챙 겼 고 ,   일 부 는   다 른   부 서 의   동 료 나   가 족   등 에 게 도   정 보 를   전 달 했 다 . [SEP] [CLS] 이   과 정 에 서 도   6 1 억 원   상 당 의   부 당   이 득 이   발 생 했 다 . [SEP] [

In [462]:
src_subtokens = tokenizer.tokenize(text)
src_subtokens

['▁',
 'b',
 '▁',
 'n',
 '▁',
 'k',
 '▁경',
 '▁남',
 '▁',
 '은',
 '▁행',
 '▁간',
 '▁부',
 '▁',
 '의',
 '▁5',
 '▁6',
 '▁2',
 '▁',
 '억',
 '▁원',
 '▁',
 '횡',
 '▁',
 '령',
 '▁사',
 '▁건',
 '▁이',
 '▁채',
 '▁',
 '잊',
 '▁',
 '히',
 '▁기',
 '▁',
 '도',
 '▁전',
 '▁인',
 '▁데',
 '▁또',
 '▁다',
 '▁',
 '른',
 '▁금',
 '▁',
 '융',
 '▁사',
 '▁고',
 '▁',
 '가',
 '▁연',
 '▁발',
 '▁하',
 '▁고',
 '▁있',
 '▁다',
 '▁',
 '.',
 '[SEP]',
 '[CLS]',
 '▁수',
 '▁법',
 '▁',
 '도',
 '▁거',
 '▁',
 '래',
 '▁기',
 '▁업',
 '▁',
 '의',
 '▁미',
 '▁공',
 '▁개',
 '▁정',
 '▁보',
 '▁',
 '로',
 '▁주',
 '▁식',
 '▁',
 '을',
 '▁거',
 '▁',
 '래',
 '▁해',
 '▁부',
 '▁당',
 '▁이',
 '▁',
 '득',
 '▁',
 '을',
 '▁취',
 '▁하',
 '▁고',
 '▁',
 ',',
 '▁고',
 '▁',
 '객',
 '▁몰',
 '▁',
 '래',
 '▁계',
 '▁좌',
 '▁',
 '를',
 '▁개',
 '▁설',
 '▁하',
 '▁',
 '는',
 '▁등',
 '▁눈',
 '▁앞',
 '▁',
 '의',
 '▁이',
 '▁익',
 '▁',
 '을',
 '▁위',
 '▁해',
 '▁온',
 '▁갖',
 '▁불',
 '▁법',
 '▁',
 '·',
 '▁편',
 '▁법',
 '▁행',
 '▁위',
 '▁',
 '가',
 '▁동',
 '▁원',
 '▁되',
 '▁고',
 '▁있',
 '▁다',
 '▁',
 '.',
 '[SEP]',
 '[CLS]',
 '▁',
 'k',
 '▁',
 'b',
 '▁국',
 

In [390]:
vocab = tokenizer.get_vocab()
print(vocab.get('▁('))
print([tokenizer.encode(token) for token in src_subtokens])

522
[[2, 522, 3], [2, 2718, 3], [2, 3524, 3], [2, 631, 3], [2, 3332, 3], [2, 4984, 3], [2, 1541, 3], [2, 2929, 3], [2, 3], [2, 517, 40, 3], [2, 3647, 3], [2, 4213, 3], [2, 3376, 3], [2, 1258, 3], [2, 3886, 3], [2, 631, 3], [2, 2554, 3], [2, 3], [2, 517, 6115, 3], [2, 4360, 3], [2, 5112, 3], [2, 2856, 3], [2, 3], [2, 517, 5859, 3], [2, 3], [2, 517, 6079, 3], [2, 3], [2, 517, 7930, 3], [2, 1725, 3], [2, 4955, 3], [2, 5136, 3], [2, 953, 3], [2, 3], [2, 517, 6896, 3], [2, 3996, 3], [2, 3612, 3], [2, 4924, 3], [2, 3], [2, 517, 6197, 3], [2, 605, 3], [2, 3], [2, 517, 6858, 3], [2, 3], [2, 517, 5712, 3], [2, 3], [2, 517, 7088, 3], [2, 2643, 3], [2, 3093, 3], [2, 3437, 3], [2, 3], [2, 517, 7996, 3], [2, 1958, 3], [2, 1875, 3], [2, 3206, 3], [2, 3], [2, 517, 7673, 3], [2, 2333, 3], [2, 4773, 3], [2, 993, 3], [2, 3533, 3], [2, 3], [2, 517, 7095, 3], [2, 3647, 3], [2, 1369, 3], [2, 3], [2, 517, 5859, 3], [2, 5049, 3], [2, 3969, 3], [2, 4360, 3], [2, 5022, 3], [2, 1763, 3], [2, 993, 3], [2, 3854, 

In [393]:
src_subtokens[0]

'▁('

In [463]:
src_subtokens = src_subtokens[:510]  ## 512가 최대인데 [SEP], [CLS] 2개 때문에 510
len(src_subtokens)

510

In [464]:
src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']
len(src_subtokens)

512

In [465]:
src_subtoken_idxs = tokenizer.convert_tokens_to_ids(src_subtokens)
src_subtoken_idxs

[2,
 517,
 380,
 517,
 425,
 517,
 415,
 953,
 1409,
 517,
 7086,
 5022,
 777,
 2423,
 517,
 7095,
 611,
 617,
 553,
 517,
 6858,
 3533,
 517,
 7964,
 517,
 6077,
 2573,
 881,
 3647,
 4451,
 517,
 7145,
 517,
 7996,
 1258,
 517,
 5859,
 4012,
 3758,
 1706,
 1861,
 1562,
 517,
 6115,
 1235,
 517,
 7073,
 2573,
 993,
 517,
 5330,
 3332,
 2235,
 4924,
 993,
 3854,
 1562,
 517,
 54,
 3,
 2,
 2872,
 2322,
 517,
 5859,
 862,
 517,
 6023,
 1258,
 3260,
 517,
 7095,
 2149,
 1023,
 835,
 4092,
 2355,
 517,
 6079,
 4213,
 3006,
 517,
 7088,
 862,
 517,
 6023,
 4998,
 2423,
 1618,
 3647,
 517,
 5926,
 517,
 7088,
 4598,
 4924,
 993,
 517,
 46,
 993,
 517,
 5370,
 2081,
 517,
 6023,
 980,
 4211,
 517,
 6116,
 835,
 2769,
 4924,
 517,
 5760,
 1815,
 1535,
 3184,
 517,
 7095,
 3647,
 3757,
 517,
 7088,
 3552,
 4998,
 3437,
 825,
 2485,
 2322,
 517,
 478,
 4832,
 2322,
 5022,
 3552,
 517,
 5330,
 1741,
 3533,
 1763,
 993,
 3854,
 1562,
 517,
 54,
 3,
 2,
 517,
 415,
 517,
 380,
 1132,
 2169,
 517,
 7

In [469]:
len(text) # 원문 문장 개수

20

In [466]:
src_subtoken_idxs.count(2)  # 2는 [CLS] 토큰
# 즉 9개의 문장에 해당하는 [CLS] 토큰만 입력된다는 이야기.

9

In [ ]:
_segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
segments_ids = []
for i, s in enumerate(segs):
    if (i % 2 == 0):
        segments_ids += s * [0]
    else:
        segments_ids += s * [1]
cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]

In [351]:
text = '''
일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다.

정부 대변인인 가토 가쓰노부(加藤勝信) 관방장관은 이날 오전 정례 기자회견에서 관련 질문에 “말씀하신 보도와 같은 사실이 없는 것으로 안다”고 밝혔다.

앞서 요미우리는 한국 측이 도쿄올림픽을 계기로 한 문 대통령의 방일을 타진했고, 일본 측은 수용하는 방향이라고 이날 보도했다.

한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다.

가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다.

그는 한국 측의 독도방어훈련에 ‘어떤 대항 조치를 생각하고 있느냐’는 질문에는 “한국 해군의 훈련에 대해 정부로서는 강한 관심을 가지고 주시하는 상황이어서 지금 시점에선 논평을 삼가겠다”고 말을 아꼈다.

가토 장관은 “다케시마(竹島·일본이 주장하는 독도의 명칭)는 역사적 사실에 비춰봐도, 국제법상으로도 명백한 일본 고유의 영토”라며 독도 영유권 주장을 되풀이했다.

그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다.
'''

In [331]:
data = list(filter(None, text.split('\n')))
data

['일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다.',
 '정부 대변인인 가토 가쓰노부(加藤勝信) 관방장관은 이날 오전 정례 기자회견에서 관련 질문에 “말씀하신 보도와 같은 사실이 없는 것으로 안다”고 밝혔다.',
 '앞서 요미우리는 한국 측이 도쿄올림픽을 계기로 한 문 대통령의 방일을 타진했고, 일본 측은 수용하는 방향이라고 이날 보도했다.',
 '한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다.',
 '가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다.',
 '그는 한국 측의 독도방어훈련에 ‘어떤 대항 조치를 생각하고 있느냐’는 질문에는 “한국 해군의 훈련에 대해 정부로서는 강한 관심을 가지고 주시하는 상황이어서 지금 시점에선 논평을 삼가겠다”고 말을 아꼈다.',
 '가토 장관은 “다케시마(竹島·일본이 주장하는 독도의 명칭)는 역사적 사실에 비춰봐도, 국제법상으로도 명백한 일본 고유의 영토”라며 독도 영유권 주장을 되풀이했다.',
 '그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다.']

In [352]:
class BertData():
    def __init__(self):
        self.tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert", do_lower_case=True)

        self.sep_token = '[SEP]'
        self.cls_token = '[CLS]'
        self.pad_token = '[PAD]'
        self.sep_vid = self.tokenizer.token2idx[self.sep_token]
        self.cls_vid = self.tokenizer.token2idx[self.cls_token]
        self.pad_vid = self.tokenizer.token2idx[self.pad_token]

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [' '.join(s) for s in src]

        labels = [0] * len(src)
        # for l in oracle_ids:
        #     labels[l] = 1

        idxs = [i for i, s in enumerate(src) if (len(s) > 1)]

        src = [src[i][:1000] for i in idxs]
        #labels = [labels[i] for i in idxs]
        src = src[:2000]
        #labels = labels[:self.args.max_nsents]

        # if (len(src) < 1):
        #     return None
        # if (len(labels) == 0):
        #     return None

        src_txt = [' '.join(sent) for sent in src]
        # text = [' '.join(ex['src_txt'][i].split()[:self.args.max_src_ntokens]) for i in idxs]
        # text = [_clean(t) for t in text]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = self.tokenizer.tokenize(text)
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        #labels = labels[:len(cls_ids)]

        #tgt_txt = '<q>'.join([' '.join(tt) for tt in tgt])
        src_txt = [original_src_txt[i] for i in idxs]
        return src_subtoken_idxs, segments_ids, cls_ids, src_txt

In [353]:
def txt2input(text):
    data = list(filter(None, text.split('\n')))
    #data = split_sentences(text)
    bertdata = BertData()
    txt_data = bertdata.preprocess(data)
    data_dict = {"src":txt_data[0],
                "segs":txt_data[1],
                "clss":txt_data[2],
                "src_txt":txt_data[3]}
    input_data = []
    input_data.append(data_dict)
    return input_data

In [357]:
input_data = txt2input(text)
input_data[0]['src'].count(2)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


6

In [354]:
inputs = txt2input(text)
len(inputs[0]['src']), len(inputs[0]['clss']), len(inputs[0]['segs'])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


(512, 6, 512)

In [199]:
device = torch.device("cuda")

def _pad(data, pad_id, width=-1):
    if (width == -1):
        width = max(len(d) for d in data)
    rtn_data = [d + [pad_id] * (width - len(d)) for d in data]
    return rtn_data

pre_src = [x['src'] for x in inputs]
pre_segs = [x['segs'] for x in inputs]
pre_clss = [x['clss'] for x in inputs]

src = torch.tensor(_pad(pre_src, 0))
segs = torch.tensor(_pad(pre_segs, 0))
mask_src = ~(src == 0)

clss = torch.tensor(_pad(pre_clss, -1))
mask_cls = ~(clss == -1)
clss[clss == -1] = 0

clss.to(device).long()
mask_cls.to(device).long()
segs.to(device).long()
mask_src.to(device).long()

# checkpoint = torch.load("D:/KoBertSum/ext/models/model_step_26000.pt")
# model = ExtSummarizer(args, device, checkpoint)
# model.eval()

# with torch.no_grad():
#     sent_scores, mask = model(src, segs, clss, mask_src, mask_cls)
#     print(sent_scores)
#     sent_scores = sent_scores + mask.float()
#     sent_scores = sent_scores.cpu().data.numpy()
#     print(sent_scores)
#     selected_ids = np.argsort(-sent_scores, 1)
#     print(selected_ids)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1

In [202]:
model = BertModel.from_pretrained("monologg/kobert")
top_vec = model(input_ids=src, token_type_ids=segs, attention_mask=mask_src)[0]
top_vec

tensor([[[-0.2849,  0.0282,  0.2609,  ..., -0.0735, -0.0258,  0.3375],
         [ 0.0772, -0.1348, -0.0158,  ...,  0.0387, -0.1104,  0.0541],
         [ 0.0997, -0.3027,  0.2173,  ..., -0.0441, -0.0477,  0.2517],
         ...,
         [ 0.0992, -0.3319,  0.0847,  ...,  0.0129,  0.1108,  0.2435],
         [ 0.0923, -0.2898,  0.2338,  ..., -0.3074,  0.1949, -0.0939],
         [-0.0438, -0.2441, -0.1525,  ..., -0.3074, -0.3792,  0.0161]]],
       grad_fn=<NativeLayerNormBackward0>)

In [217]:
src.size(), clss.size(), segs.size()

(torch.Size([1, 512]), torch.Size([1, 6]), torch.Size([1, 512]))

In [214]:
clss

tensor([[  0, 110, 290, 374, 432, 486]])

In [213]:
top_vec[torch.arange(top_vec.size(0)).unsqueeze(1)]

tensor([[[[-0.2849,  0.0282,  0.2609,  ..., -0.0735, -0.0258,  0.3375],
          [ 0.0772, -0.1348, -0.0158,  ...,  0.0387, -0.1104,  0.0541],
          [ 0.0997, -0.3027,  0.2173,  ..., -0.0441, -0.0477,  0.2517],
          ...,
          [ 0.0992, -0.3319,  0.0847,  ...,  0.0129,  0.1108,  0.2435],
          [ 0.0923, -0.2898,  0.2338,  ..., -0.3074,  0.1949, -0.0939],
          [-0.0438, -0.2441, -0.1525,  ..., -0.3074, -0.3792,  0.0161]]]],
       grad_fn=<IndexBackward0>)

In [207]:
sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
sents_vec

tensor([[[-0.2849,  0.0282,  0.2609,  ..., -0.0735, -0.0258,  0.3375],
         [-0.0033, -0.2608, -0.1116,  ..., -0.0259, -0.1240,  0.1610],
         [-0.0964, -0.1890,  0.0300,  ..., -0.0450, -0.0738,  0.2506],
         [-0.1293, -0.1998,  0.0067,  ..., -0.0116, -0.0832,  0.2335],
         [-0.0662, -0.2018,  0.0209,  ..., -0.0581, -0.0892,  0.2462],
         [-0.0804, -0.2358, -0.0202,  ..., -0.0204, -0.1140,  0.1821]]],
       grad_fn=<IndexBackward0>)

In [ ]:
sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()
        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)